## Visualization four arms interferometer

This notebook refers to the visualization of the results of the ``four_interferometer.py`` script, which reproduces the model of https://arxiv.org/abs/2209.00671. We plot the mean and the median erros for a random strategy only.

In [1]:
import numpy as np
import pandas as pd
import qsensoropt.plot_utils as pu
import matplotlib.pyplot as plt
from math import pi
from os.path import join
from os import makedirs

2023-08-17 14:35:22.845607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 14:35:23.008958: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-17 14:35:23.008980: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-17 14:35:23.998713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
########################
batchsize = 32
num_steps = 100
num_particles = 24000
static = False
########################

In [ ]:
def generate_sim_name(
    batchsize, num_steps, num_particles,
    median: bool = False, mean: bool = False,
    random: bool = False, static: bool = False,
):
    name = f'four_interferometer_num_part_{num_particles}'
    name += '_static' if static else ''
    name += '_random' if random else ''
    name += '_median' if median else ''
    name += '_mean' if mean else ''
    name += f'_batchsize_{batchsize}_num_steps_{num_steps}_max_resources_{num_steps:.2f}'
    name += f'_ll_False_cl_True'
    return name

In [ ]:
name = generate_sim_name(
    batchsize, num_steps, num_particles,
)
name_median_rnd = generate_sim_name(
    batchsize, num_steps, num_particles,
    random=True, median=True,
)
name_mean_rnd = generate_sim_name(
    batchsize, num_steps, num_particles,
    random=True, mean=True,
)
if static:
    name_median_static = generate_sim_name(
        batchsize, num_steps, num_particles,
        random=False, median=True, static=True,
    )
    name_mean_static = generate_sim_name(
        batchsize, num_steps, num_particles,
        random=False, mean=True, static=True,
    )

In [ ]:
prec_median_rnd_df = pd.read_csv(
    f'./four_interferometer/data/{name_median_rnd}_eval.csv',
)
prec_mean_rnd_df = pd.read_csv(
    f'./four_interferometer/data/{name_mean_rnd}_eval.csv',
)
if static:
    prec_median_static_df = pd.read_csv(
    f'./four_interferometer/data/{name_median_static}_eval.csv',
    )
    prec_mean_static_df = pd.read_csv(
        f'./four_interferometer/data/{name_mean_static}_eval.csv',
    )

In [ ]:
prec_median_rnd_df["NumShot"] = prec_median_rnd_df["Resources"]
prec_mean_rnd_df["NumShot"] = prec_mean_rnd_df["Resources"]
if static:
    prec_median_static_df["NumShot"] = prec_median_static_df["Resources"]
    prec_mean_static_df["NumShot"] = prec_mean_static_df["Resources"]

In [ ]:
prec_median_rnd_df["Error"] = prec_median_rnd_df["MedianSE"]
prec_mean_rnd_df["Error"] = prec_mean_rnd_df["MSE"]
if static:
    prec_median_static_df["Error"] = prec_median_static_df["MedianSE"]
    prec_mean_static_df["Error"] = prec_mean_static_df["MSE"]

In [ ]:
xaxis = np.linspace(1, num_steps, num_steps)

In [ ]:
prec_cr_df = pd.DataFrame({"NumShot": xaxis, "Error": 2.5/xaxis})

In [ ]:
img_dir = f'./four_interferometer/images/{name}'
makedirs(img_dir, exist_ok=True)

In [ ]:
if static:
    pu.plot_multiples(
            [prec_mean_rnd_df, prec_median_rnd_df,
             prec_mean_static_df, prec_median_static_df, prec_cr_df], 
            "NumShot", "Error", 
            ["Mean Rnd", "Median Rnd",
             "Mean Static", "Median Static", "CRbound"],
            figsize=(3, 2), dpi=200,
            #title="Precision",
            path=img_dir,
        )
else:
    pu.plot_multiples(
        [prec_mean_rnd_df, prec_median_rnd_df, prec_cr_df], 
        "NumShot", "Error", 
        ["Mean", "Median", "CRbound"],
        figsize=(3, 2), dpi=200,
        #title="Precision",
        path=img_dir,
    )

### Joint plot

In [ ]:
img_dir_all = f'./four_interferometer/images/'
makedirs(img_dir, exist_ok=True)

In [ ]:
list_num_particles = [8192, 16384, 32768]

In [ ]:
list_median_rnd_name = [
    generate_sim_name(
    batchsize, num_steps, np,
    random=True, median=True,)
    for np in list_num_particles
]
list_mean_rnd_name = [
    generate_sim_name(
    batchsize, num_steps, np,
    random=True, mean=True,)
    for np in list_num_particles
]

In [ ]:
list_median_rnd_prec = [
    pd.read_csv(
        f'./four_interferometer/data/{name}_eval.csv',
    ) for name in list_median_rnd_name
]
list_mean_rnd_prec = [
    pd.read_csv(
        f'./four_interferometer/data/{name}_eval.csv',
    ) for name in list_mean_rnd_name
]

In [ ]:
for prec in list_median_rnd_prec:
    prec["NumShot"] = prec["Resources"]
for prec in list_mean_rnd_prec:
    prec["NumShot"] = prec["Resources"]
for prec in list_median_rnd_prec:
    prec["Error"] = prec["MedianSE"]
for prec in list_mean_rnd_prec:
    prec["Error"] = prec["MSE"]

In [ ]:
# List of names for the legend
list_legend_median = [
    f"Median NumPart={np}" for np in list_num_particles
]
list_legend_mean = [
    f"Mean NumPart={np}" for np in list_num_particles
]

In [ ]:
pu.plot_multiples(
        list_median_rnd_prec+list_mean_rnd_prec, 
        "NumShot", "Error", 
        list_legend_median+list_legend_mean,
        figsize=(3, 2), dpi=200,
        #title="Precision",
        path=img_dir_all,
    )